In [10]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
import os
import yaml
import itertools
import shutil
from datetime import datetime
import sys

In [11]:
# ##SAVE: NEEM INTERP
# sv_links = pd.read_excel('/Users/quinnmackay/Documents/GitHub/BICC/Data Storage/Tiepoints/GISP2-NEEM.xlsx', sheet_name=2, skiprows=1, usecols=[1,2], names=['GISP2', 'NEEM2011'])

# NEEM_interp = pd.read_excel('/Users/quinnmackay/Documents/GitHub/BICC/Data Storage/Tiepoints/GICC21 matchpoints.xlsx', sheet_name=2, skiprows=1, usecols=[9,15], names=['NEEM_og', 'NEEM2011'])
# NEEM_interp.dropna(inplace=True)

# sv_links = sv_links[sv_links['NEEM2011'] <= NEEM_interp['NEEM2011'].max()]
# sv_links['NEEM'] = np.interp(sv_links['NEEM2011'], NEEM_interp['NEEM2011'], NEEM_interp['NEEM_og'])

# sv_links = sv_links[['GISP2', 'NEEM']]

# folder='GL_20Cols'

In [12]:
reference = 'GICC05_Revised'
comment_add = 'GICC05 revised tiepoints from accompanying datafile for Rasmussen et al. 2014 and Seierstad et al. 2014'

sv_links = pd.read_excel('/Users/quinnmackay/Documents/GitHub/BICC/Data Storage/Tiepoints/GICC05 revised match points and d18O and CH4.xlsx', sheet_name=1, skiprows=26, usecols=[1,2], names=['NG1', 'NG2'])
sv_links.dropna(inplace=True)

folder='Greenland'

sv_links

,NG1,NG2
285,1347.744,1347.301
286,1347.983,1347.572
287,1348.450,1347.982
288,1349.055,1348.622
289,1349.340,1348.910
290,1352.732,1352.313
291,1354.514,1354.108
292,1354.832,1354.471
293,1356.074,1355.647
294,1356.560,1356.152


In [13]:
#load cores list from params
params = f'/Users/quinnmackay/Documents/GitHub/BICC/Antarctic Chronology Accuracy Project/{folder}/parameters.yml'
with open(params, 'r') as f:
    first_line = f.readline()
params_load = yaml.safe_load(first_line)
list_sites = params_load['list_sites']

# get all link combos
pairs = [f"{a}-{b}" for a, b in itertools.combinations(list_sites, 2)]

#get all combos possible from the svensson links
subset = list(sv_links.columns)
valid_pairs = [p for p in pairs if all(site in subset for site in p.split('-'))]

In [14]:
#now, create all the link files
sv_synchros = {}
sv_comments = {}
for pair in valid_pairs:
    site_a, site_b = pair.split('-')

    # Drop NaN rows to get only valid shared tiepoints
    sv_synchros[pair] = sv_links[[site_a, site_b]].dropna()
    sv_synchros[pair].columns = ['depth1', 'depth2']
    sv_synchros[pair]['reference'] = reference  # Load reference string
    sv_comments[pair] = [comment_add]

#now load existing synchros
os.chdir(f'/Users/quinnmackay/Documents/GitHub/BICC/Antarctic Chronology Accuracy Project/{folder}')
existing_synchros = {}

for pair in valid_pairs:

    folder_path = pair
    file_path = os.path.join(folder_path, f"{reference}_{pair}.txt")
    os.makedirs(folder_path, exist_ok=True)

    if os.path.exists(file_path):
        raise FileExistsError(f"File already exists: {file_path}")

    combined = sv_synchros[pair].copy(deep=True) # svensson links load
    new_comments = sv_comments.get(pair, [])

    with open(file_path, 'w') as f:
        # Write new comments first
        for line in new_comments:
            f.write(f"#{line}\n")
        # Write merged DataFrame below
        combined.to_csv(f, sep='\t', index=False)

    print(f"Saving new synchro to {file_path}")

Saving new synchro to NG1-NG2/GICC05_Revised_NG1-NG2.txt
